In [1]:
!pip install torchdata==0.7.1 --quiet
!pip install dgl -f https://data.dgl.ai/wheels/torch-2.1/cu121/repo.html --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.4/184.4 kB 4.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtune 0.6.1 requires torchdata==0.11.0, but you have torchdata 0.7.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 483.2/483.2 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 115.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.8 MB/s eta 0:00:00
   ━━━━━

In [2]:
!pip install tensorboardX pandas numpy networkx tqdm scikit-learn scipy --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 2.6 MB/s eta 0:00:00


In [3]:
# -*- coding: utf-8 -*-
"""
AI-GFACN Implementation for Accident Prediction
Integrated with DSTGCN Data Loading Pipeline & Utilities
"""

import os
import sys
import json
import shutil
import copy
import time
import datetime
import math
import warnings
import pickle
from typing import List, Dict, Tuple

import numpy as np
import pandas as pd
import networkx as nx
from tqdm import tqdm
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# DGL Imports
import dgl
from dgl.nn.pytorch import GraphConv

# Sklearn & Metrics
from sklearn import metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Tensorboard
from tensorboardX import SummaryWriter

# Ignore warnings
warnings.filterwarnings('ignore')


# ==========================================
# 1. CONFIGURATION
# ==========================================

config_content = '''{
  "model_name": "AI_GFACN",
  "train_repeat_times": 1,
  "epochs": 100,
  "cuda": 0,
  "temporal_features_number": 1,
  "weather_features_number": 38,
  "external_temporal_features_number": 5,
  "poi_features_number": 22,
  "batch_size": 64,
  "learning_rate": 1e-3,
  "loss_function": "bce_loss",
  "optim": "Adam",
  "weight_decay": 1e-3,
  "K_hop": 10,
  "spatial_features_mean": [
    6.40254292e+00, 4.81553347e+00, 2.64811501e+01, 5.54260681e+00, 7.84915710e+00,
    9.01424670e+00, 1.05751666e+00, 2.83014290e+01, 1.77408007e+01, 3.24747843e+01,
    1.05794121e+00, 3.47343126e+00, 6.26610390e+00, 3.39390529e+00, 1.28002351e+01,
    5.71796174e-02, 2.36710361e-02, 5.17406913e-02, 8.18431583e-01, 4.57574675e-01,
    1.61370302e+02, 2.00511484e+00
  ],
  "spatial_features_std": [
    8.25421040e+00, 6.98772605e+00, 3.12181863e+01, 7.78804699e+00, 1.12482891e+01,
    8.86203753e+00, 4.88544842e+00, 4.65677369e+01, 1.88312515e+01, 6.70090744e+01,
    2.95457633e+00, 4.29173854e+00, 9.78727429e+00, 3.76843985e+00, 1.19405955e+01,
    2.71230076e-01, 1.88342431e-01, 2.82777835e-01, 2.18729099e+00, 1.55007715e+00,
    2.04879612e+02, 7.87830458e-02
  ],
  "temporal_features_mean": [18.54884516],
  "temporal_features_std": [19.87195773],
  "external_features_mean": [
    7.02588369e+01, 5.28922463e+01, 5.95480331e+01, 2.98048133e+01, 6.23360889e+00,
    7.17909065e+01, 1.31128848e-02, 8.48346636e-01, 4.86031927e-02, 1.65336374e-02,
    1.71037628e-03, 1.71037628e-02, 1.51083238e-02, 7.41163056e-03, 2.28050171e-03,
    1.58209806e-02, 1.25427594e-02, 0.00000000e+00, 1.42531357e-04, 1.28278221e-03,
    2.10946408e-02, 5.23090080e-02, 2.53705815e-02, 4.53249715e-02, 1.62058153e-01,
    1.96693273e-02, 4.53249715e-02, 6.01482326e-02, 6.14310148e-02, 5.60148233e-02,
    6.07183580e-02, 5.91505131e-02, 1.58209806e-02, 2.28050171e-03, 2.29618016e-01,
    2.87913341e-02, 5.11687571e-02, 3.70581528e-03, 8.90992018e+00, 1.67015393e+01,
    2.72548461e+00, 1.22075257e+01, 1.92702395e-01
  ],
  "external_features_std": [
    1.45939773e+01, 1.86117750e+01, 2.39727858e+01, 2.15191858e-01, 5.02840584e+00,
    1.71451636e+01, 1.05974340e-01, 3.50647069e-01, 2.05374727e-01, 1.19435494e-01,
    2.91935396e-02, 1.22304197e-01, 1.10326636e-01, 6.60570950e-02, 3.95302952e-02,
    1.18032057e-01, 9.83734863e-02, 1.00000000e+00, 8.44069685e-03, 2.52931923e-02,
    1.17508093e-01, 1.82744266e-01, 1.15431393e-01, 1.69085448e-01, 3.23940225e-01,
    9.79301412e-02, 1.64816793e-01, 1.96033008e-01, 2.05755011e-01, 1.93423139e-01,
    2.05099602e-01, 1.97963022e-01, 1.01824764e-01, 3.76843851e-02, 3.46547020e-01,
    1.50627796e-01, 1.96944453e-01, 4.76105938e-02, 8.32550240e-01, 8.70795832e+00,
    1.86828552e+00, 5.96074486e+00, 3.94421325e-01
  ]
}'''

# Write config to file
with open("config.json", "w") as f:
    f.write(config_content)

# Load config
with open("config.json") as file:
    config = json.load(file)

def get_attribute(name, defaultValue=None):
    try:
        return config[name]
    except KeyError:
        return defaultValue

# Set Device
config['device'] = f'cuda:{get_attribute("cuda")}' if torch.cuda.is_available() and get_attribute("cuda") >= 0 else 'cpu'
print(f"Using device: {config['device']}")


# ==========================================
# 2. UTILS & COORD TRANSFORM (From DSTGCN)
# ==========================================

x_pi = 3.14159265358979324 * 3000.0 / 180.0
pi = 3.1415926535897932384626
a = 6378245.0
ee = 0.00669342162296594323

def gcj02_to_bd09(lng, lat):
    z = math.sqrt(lng * lng + lat * lat) + 0.00002 * math.sin(lat * x_pi)
    theta = math.atan2(lat, lng) + 0.000003 * math.cos(lng * x_pi)
    bd_lng = z * math.cos(theta) + 0.0065
    bd_lat = z * math.sin(theta) + 0.006
    return [bd_lng, bd_lat]

def bd09_to_gcj02(bd_lon, bd_lat):
    x = bd_lon - 0.0065
    y = bd_lat - 0.006
    z = math.sqrt(x * x + y * y) - 0.00002 * math.sin(y * x_pi)
    theta = math.atan2(y, x) - 0.000003 * math.cos(x * x_pi)
    gg_lng = z * math.cos(theta)
    gg_lat = z * math.sin(theta)
    return [gg_lng, gg_lat]

def wgs84_to_gcj02(lng, lat):
    if out_of_china(lng, lat):
        return [lng, lat]
    dlat = _transformlat(lng - 105.0, lat - 35.0)
    dlng = _transformlng(lng - 105.0, lat - 35.0)
    radlat = lat / 180.0 * pi
    magic = math.sin(radlat)
    magic = 1 - ee * magic * magic
    sqrtmagic = math.sqrt(magic)
    dlat = (dlat * 180.0) / ((a * (1 - ee)) / (magic * sqrtmagic) * pi)
    dlng = (dlng * 180.0) / (a / sqrtmagic * math.cos(radlat) * pi)
    mglat = lat + dlat
    mglng = lng + dlng
    return [mglng, mglat]

def _transformlat(lng, lat):
    ret = -100.0 + 2.0 * lng + 3.0 * lat + 0.2 * lat * lat + \
          0.1 * lng * lat + 0.2 * math.sqrt(math.fabs(lng))
    ret += (20.0 * math.sin(6.0 * lng * pi) + 20.0 *
            math.sin(2.0 * lng * pi)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lat * pi) + 40.0 *
            math.sin(lat / 3.0 * pi)) * 2.0 / 3.0
    ret += (160.0 * math.sin(lat / 12.0 * pi) + 320 *
            math.sin(lat * pi / 30.0)) * 2.0 / 3.0
    return ret

def _transformlng(lng, lat):
    ret = 300.0 + lng + 2.0 * lat + 0.1 * lng * lng + \
          0.1 * lng * lat + 0.1 * math.sqrt(math.fabs(lng))
    ret += (20.0 * math.sin(6.0 * lng * pi) + 20.0 *
            math.sin(2.0 * lng * pi)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lng * pi) + 40.0 *
            math.sin(lng / 3.0 * pi)) * 2.0 / 3.0
    ret += (150.0 * math.sin(lng / 12.0 * pi) + 300.0 *
            math.sin(lng / 30.0 * pi)) * 2.0 / 3.0
    return ret

def out_of_china(lng, lat):
    return not (lng > 73.66 and lng < 135.05 and lat > 3.86 and lat < 53.55)

def convert_by_type(lng, lat, type):
    if type == 'g2b': return gcj02_to_bd09(lng, lat)
    elif type == 'b2g': return bd09_to_gcj02(lng, lat)
    elif type == 'w2g': return wgs84_to_gcj02(lng, lat)
    elif type == 'g2w':
        return [lng, lat]
    else: return [lng, lat]

def convert_to_gpu(data):
    return data.to(get_attribute('device'))

def convert_train_truth_to_gpu(train_data, truth_data):
    train_data = [convert_to_gpu(data) for data in train_data]
    truth_data = convert_to_gpu(truth_data)
    return train_data, truth_data

def load_model(model, modelFilePath):
    model.load_state_dict(torch.load(modelFilePath))
    return model

def save_model(path: str, **save_dict):
    os.makedirs(os.path.split(path)[0], exist_ok=True)
    torch.save(save_dict, path)


# ==========================================
# 3. METRICS & LOSS (Added MAE, RMSE, PCC)
# ==========================================

class MSELoss(nn.Module):
    def __init__(self):
        super(MSELoss, self).__init__()
        self.mse_loss = nn.MSELoss(reduction='sum')
    def forward(self, truth, predict):
        return self.mse_loss(predict, truth)

class BCELoss(nn.Module):
    def __init__(self):
        super(BCELoss, self).__init__()
        self.bce_loss = nn.BCELoss()
    def forward(self, truth, predict):
        return self.bce_loss(predict, truth)

def evaluate(y_predictions: np.ndarray, y_targets: np.ndarray, threshold: float = 0.5):
    if y_predictions.shape != y_targets.shape:
        y_predictions = y_predictions.reshape(y_targets.shape)

    # --- 1. Regression Scale Metrics (Requested) ---
    rmse = mean_squared_error(y_targets, y_predictions) ** 0.5
    mae = mean_absolute_error(y_targets, y_predictions)
    
    # Pearson Correlation Coefficient
    # Flatten mảng để tính tương quan
    pcc, _ = pearsonr(y_predictions.flatten(), y_targets.flatten())

    # --- 2. Classification Metrics (Original) ---
    y_predictions_class = y_predictions >= threshold
    y_targets_class = y_targets == 1

    tp = ((y_predictions_class == 1) & (y_targets_class == 1)).sum()
    fp = ((y_predictions_class == 1) & (y_targets_class == 0)).sum()
    fn = ((y_predictions_class == 0) & (y_targets_class == 1)).sum()

    precision = tp / (tp + fp) if (tp + fp) != 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) != 0 else 0.0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0.0
    
    try:
        auc = metrics.roc_auc_score(y_targets_class, y_predictions)
    except ValueError:
        auc = 0.0

    results = {
        'MAE': mae,      # Added
        'RMSE': rmse,    # Added
        'PCC': pcc,      # Added
        'PRECISION': precision,
        'RECALL': recall,
        'F1-SCORE': f1_score,
        'AUC': auc
    }
    return results


# ==========================================
# 4. MODEL ARCHITECTURE (AI-GFACN)
# ==========================================

class SpatialTemporalEmbedding(nn.Module):
    """
    STE Module: Tạo embedding từ đặc trưng không gian (Node2Vec style) 
    và thời gian (Speed + Positional Encoding).
    """
    def __init__(self, spatial_dim, temporal_dim, hidden_dim, time_steps):
        super(SpatialTemporalEmbedding, self).__init__()
        
        # Chiếu đặc trưng không gian (22 features)
        self.spatial_proj = nn.Linear(spatial_dim, hidden_dim)
        
        # Chiếu đặc trưng thời gian (1 feature - tốc độ)
        self.temporal_proj = nn.Linear(temporal_dim, hidden_dim)
        
        # Positional Encoding cho Time steps (24 giờ)
        self.time_emb = nn.Parameter(torch.zeros(1, time_steps, hidden_dim))
        nn.init.xavier_uniform_(self.time_emb)

    def forward(self, spatial_feat, temporal_feat):
        # spatial_feat: [N, S_dim] -> [N, 1, Hidden]
        s_emb = self.spatial_proj(spatial_feat).unsqueeze(1)
        
        # temporal_feat: [N, T, T_dim] -> [N, T, Hidden]
        t_val_emb = self.temporal_proj(temporal_feat)
        
        # Cộng Positional Encoding: t_pos = t_val + P
        t_pos_emb = t_val_emb + self.time_emb[:, :temporal_feat.size(1), :]
        
        # Hợp nhất: e = ES + ET (theo mô tả AI-GFACN)
        ste = s_emb + t_pos_emb 
        return ste # [N, T, Hidden]

class ST_Attention(nn.Module):
    """
    Spatial & Temporal Attention Mechanism
    """
    def __init__(self, hidden_dim, num_heads=4):
        super(ST_Attention, self).__init__()
        self.num_heads = num_heads
        self.d_k = hidden_dim // num_heads
        
        self.W_Q = nn.Linear(hidden_dim, hidden_dim)
        self.W_K = nn.Linear(hidden_dim, hidden_dim)
        self.W_V = nn.Linear(hidden_dim, hidden_dim)
        
        self.dropout = nn.Dropout(0.1)
        self.layer_norm = nn.LayerNorm(hidden_dim)

    def forward(self, x, ste):
        # x: Input features h
        # ste: Embedding e
        
        # Query/Key kết hợp thông tin embedding
        query = x + ste
        key = x + ste
        value = x

        batch_size = x.size(0)
        
        Q = self.W_Q(query).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        K = self.W_K(key).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        V = self.W_V(value).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)

        # Scaled Dot-Product Attention
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        attn = F.softmax(scores, dim=-1)
        attn = self.dropout(attn)
        
        context = torch.matmul(attn, V) 
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.num_heads * self.d_k)
        
        return self.layer_norm(x + context)

class GatedFusion(nn.Module):
    """
    Gated Fusion: Hợp nhất feature A và B qua cổng sigmoid
    """
    def __init__(self, hidden_dim):
        super(GatedFusion, self).__init__()
        self.gate_fc = nn.Linear(hidden_dim * 2, 1)

    def forward(self, h_A, h_B):
        combined = torch.cat([h_A, h_B], dim=-1)
        z = torch.sigmoid(self.gate_fc(combined))
        return z * h_A + (1 - z) * h_B

class AI_GFACN(nn.Module):
    """
    Mô hình chính: AI-GFACN
    Cấu trúc: STE -> Attention -> GCN -> Fusion -> Output
    """
    def __init__(self, spatial_dim, temporal_dim, external_dim, hidden_dim=64):
        super(AI_GFACN, self).__init__()
        
        # 1. Embeddings
        self.ste = SpatialTemporalEmbedding(spatial_dim, temporal_dim, hidden_dim, time_steps=24)
        self.input_fc = nn.Linear(temporal_dim, hidden_dim)
        
        # 2. Attention
        self.st_attention = ST_Attention(hidden_dim, num_heads=4)
        
        # 3. GCN (Mô phỏng Encoder-Decoder spatial)
        self.gcn1 = GraphConv(hidden_dim, hidden_dim)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.gcn2 = GraphConv(hidden_dim, hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        
        # 4. Fusion
        self.fusion = GatedFusion(hidden_dim)
        
        # 5. External Features (Weather, Time metadata)
        self.ext_proj = nn.Linear(external_dim, hidden_dim)
        
        # 6. Output Layers (Binary Classification cho bài toán DSTGCN)
        self.output_fc = nn.Sequential(
            nn.Linear(hidden_dim * 2, 32),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
        self.dropout = nn.Dropout(0.3)

    def forward(self, g, spatial_feat, temporal_feat, external_feat):
        # 1. STE & Input Processing
        ste_emb = self.ste(spatial_feat, temporal_feat) 
        h_in = self.input_fc(temporal_feat)
        
        # 2. Attention
        h_att = self.st_attention(h_in, ste_emb)
        # Global Temporal Pooling (Tổng hợp 24h thành 1 vector)
        h_t_agg = torch.mean(h_att, dim=1) 
        
        # 3. GCN Layers
        h_gcn = self.gcn1(g, h_t_agg)
        h_gcn = self.bn1(h_gcn)
        h_gcn = F.relu(h_gcn)
        h_gcn = self.dropout(h_gcn)
        
        h_gcn = self.gcn2(g, h_gcn)
        h_gcn = self.bn2(h_gcn)
        h_gcn = F.relu(h_gcn)
        
        # 4. Fusion
        h_fused = self.fusion(h_gcn, h_t_agg)
        
        # 5. External Features
        ext_emb = self.ext_proj(external_feat)
        # Broadcast external features (Batch -> Nodes)
        ext_emb_expanded = []
        batch_num_nodes = g.batch_num_nodes().tolist()
        for i, num_nodes in enumerate(batch_num_nodes):
            ext_emb_expanded.append(ext_emb[i].repeat(num_nodes, 1))
        ext_emb_nodes = torch.cat(ext_emb_expanded, dim=0)
        
        # 6. Final Prediction
        final_feat = torch.cat([h_fused, ext_emb_nodes], dim=1)
        out = self.output_fc(final_feat)
        
        # Chỉ lấy output của node đầu tiên mỗi subgraph (Target node)
        output_list = []
        node_idx_offset = 0
        for num_nodes in batch_num_nodes:
            output_list.append(out[node_idx_offset])
            node_idx_offset += num_nodes
            
        return torch.stack(output_list)


# ==========================================
# 5. DATA LOADING (With Fixes)
# ==========================================

longitudeMin = 116.09608
longitudeMax = 116.71040
latitudeMin = 39.69086
latitudeMax = 40.17647

# Pre-calc geo bounds (g2w)
longitudeMin, latitudeMin = convert_by_type(lng=longitudeMin, lat=latitudeMin, type="g2w")
longitudeMax, latitudeMax = convert_by_type(lng=longitudeMax, lat=latitudeMax, type="g2w")

divideBound = 5
widthSingle = 0.01 / math.cos(latitudeMin / 180 * math.pi) / divideBound
width = math.floor((longitudeMax - longitudeMin) / widthSingle)
heightSingle = 0.01 / divideBound
height = math.floor((latitudeMax - latitudeMin) / heightSingle)

def collate_fn(batch):
    ret = list()
    for idx, item in enumerate(zip(*batch)):
        if isinstance(item[0], torch.Tensor):
            if idx < 3:
                ret.append(torch.cat(item))
            else:
                ret.append(torch.stack(item))
        elif isinstance(item[0], dgl.DGLGraph):
            ret.append(dgl.batch(item))
        else:
            raise ValueError(f'batch must contain tensors or graphs; found {type(item[0])}')
    return tuple(ret)

def fill_speed(speed_data):
    # Optimized fill_speed function
    date_range = pd.date_range(start="2018-08-01", end="2018-11-01", freq="1H")[:-1]
    speed_data = speed_data.resample(rule="1H").mean()
    
    missing_dates = []
    # Fast check for NaNs
    if speed_data.isnull().values.any():
        for date in tqdm(date_range, desc="Checking missing dates"):
            if date not in speed_data.index or speed_data.loc[date].isna().any():
                missing_dates.append(date)
    
    if len(missing_dates) > 0:
        print(f"Filling {len(missing_dates)} missing dates...")
        one_week, two_week = datetime.timedelta(days=7), datetime.timedelta(days=14)
        for date in tqdm(missing_dates, desc='Filling speed'):
            filled = False
            for offset in [one_week, -one_week, two_week, -two_week]:
                ref_date = date + offset
                if ref_date in speed_data.index and not speed_data.loc[ref_date].isna().any():
                    speed_data.loc[date] = speed_data.loc[ref_date]
                    filled = True
                    break
            if not filled:
                 # Fallback to mean if strictly necessary
                 pass
    return speed_data

class AccidentDataset(Dataset):
    def __init__(self, k_order, network, node_attr, accident, weather, speed, 
                 sf_scaler=None, tf_scaler=None, ef_scaler=None):
        self.k_order = k_order
        self.network = network
        self.nodes = node_attr
        self.accident = accident
        self.weather = weather
        self.speed = speed
        self.sf_scaler = sf_scaler
        self.tf_scaler = tf_scaler
        self.ef_scaler = ef_scaler

    def __getitem__(self, sample_id: int):
        _, _, accident_time, node_id, target = self.accident.iloc[sample_id]
        
        # Subgraph generation
        neighbors = nx.single_source_shortest_path_length(self.network, node_id, cutoff=self.k_order)
        neighbors.pop(node_id, None)
        neighbors = [node_id] + sorted(neighbors.keys())
        
        sub_graph_nx = nx.subgraph(self.network, neighbors)
        relabel_map = {old_label: new_label for new_label, old_label in enumerate(neighbors)}
        sub_graph_nx = nx.relabel_nodes(sub_graph_nx, relabel_map)
        sub_graph_nx.add_edges_from([(v, v) for v in sub_graph_nx.nodes])
        g = dgl.from_networkx(sub_graph_nx)

        # Time
        date_range = pd.date_range(end=accident_time.strftime("%Y%m%d %H"), freq="1H", periods=24)
        
        # Spatial
        selected_nodes = self.nodes.loc[neighbors]
        spatial_features = selected_nodes['spatial_features'].tolist()

        # Grid Speed Lookup
        x_ids = np.floor((selected_nodes['XCoord'].values - longitudeMin) / widthSingle).astype(np.int64)
        y_ids = np.floor((selected_nodes['YCoord'].values - latitudeMin) / heightSingle).astype(np.int64)
        
        # FIX 1: Clip coordinates to prevent negative indexing or out-of-bounds
        x_ids = np.clip(x_ids, 0, width - 1)
        y_ids = np.clip(y_ids, 0, height - 1)
        
        grid_keys = [f'{y},{x}' for y, x in zip(y_ids, x_ids)]
        
        # FIX 2: Safe Reindexing (Handle missing grid keys)
        try:
            speed_slice = self.speed.loc[date_range]
            # reindex will fill missing columns with 0
            temporal_features = speed_slice.reindex(columns=grid_keys, fill_value=0).values.transpose()
        except KeyError:
            # Fallback if date is missing
            temporal_features = np.zeros((len(grid_keys), 24))

        # External
        weather_data = self.weather.loc[date_range[-1]].tolist()
        external_features = weather_data + [accident_time.month, accident_time.day, 
                                            accident_time.dayofweek, accident_time.hour, 
                                            int(accident_time.dayofweek >= 5)]

        # Scaling
        if self.sf_scaler:
            mean, std = self.sf_scaler
            spatial_features = (np.array(spatial_features) - mean) / std
        if self.tf_scaler:
            mean, std = self.tf_scaler
            temporal_features = (np.array(temporal_features) - mean) / std
        if self.ef_scaler:
            mean, std = self.ef_scaler
            external_features = (np.array(external_features) - mean) / std

        # FIX 3: Unsqueeze(-1) to create [Nodes, Time, Features] shape for Model
        return (g, 
                torch.tensor(spatial_features).float(), 
                torch.tensor(temporal_features).unsqueeze(-1).float(), 
                torch.tensor(external_features).float(), 
                torch.tensor(target).float())

    def __len__(self):
        return len(self.accident)

def get_data_loaders(k_order, batch_size):
    # PATHS - Update these to your Kaggle/Local paths
    base_path = '/kaggle/input/dstgcn-dataset' 
    if not os.path.exists(base_path):
        base_path = 'data' # Local fallback
        
    network_path = os.path.join(base_path, 'beijing_roadnet.gpickle')
    node_attr_path = os.path.join(base_path, 'edges_data.h5')
    accident_path = os.path.join(base_path, 'accident.h5')
    weather_path = os.path.join(base_path, 'weather.h5')
    speed_path = os.path.join(base_path, 'all_grids_speed.h5')

    sf_mean = np.array(get_attribute('spatial_features_mean'))
    sf_std = np.array(get_attribute('spatial_features_std'))
    tf_mean = np.array(get_attribute('temporal_features_mean'))
    tf_std = np.array(get_attribute('temporal_features_std'))
    ef_mean = np.array(get_attribute('external_features_mean'))
    ef_std = np.array(get_attribute('external_features_std'))

    print(f"Loading network from {network_path}...")
    with open(network_path, 'rb') as f:
        network = pickle.load(f)
    
    nodes = pd.read_hdf(node_attr_path)
    weather = pd.read_hdf(weather_path)
    print("Loading speed data...")
    speed = fill_speed(pd.read_hdf(speed_path))

    dls = dict()
    for key in ['train', 'validate', 'test']:
        print(f"Loading {key} accident data...")
        accident = pd.read_hdf(accident_path, key=key)
        dataset = AccidentDataset(k_order, network, nodes, accident, weather, speed,
                                  sf_scaler=(sf_mean, sf_std),
                                  tf_scaler=(tf_mean, tf_std),
                                  ef_scaler=(ef_mean, ef_std))
        dls[key] = DataLoader(dataset=dataset,
                              batch_size=batch_size,
                              shuffle=True,
                              drop_last=False,
                              collate_fn=collate_fn,
                              num_workers=2) # Adjust workers based on environment
    return dls


# ==========================================
# 6. TRAINING FUNCTION
# ==========================================

def train_model(model: nn.Module,
                data_loaders: Dict[str, DataLoader],
                loss_func: callable,
                optimizer,
                model_folder: str,
                tensorboard_folder: str):
    phases = ['train', 'validate', 'test']
    writer = SummaryWriter(tensorboard_folder)
    num_epochs = get_attribute('epochs')

    since = time.time()
    model = convert_to_gpu(model)
    loss_func = convert_to_gpu(loss_func)

    save_dict, best_f1_score = {'model_state_dict': copy.deepcopy(model.state_dict()), 'epoch': 0}, 0
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=.5, patience=2, threshold=1e-3, min_lr=1e-6)
    test_metric = None

    try:
        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)
            running_loss = {phase: 0.0 for phase in phases}
            running_metrics = {phase: dict() for phase in phases}
            save_validate_this_epoch = False

            for phase in phases:
                if phase == 'train':
                    model.train()
                else:
                    model.eval()

                steps, predictions, targets = 0, list(), list()
                tqdm_loader = tqdm(data_loaders[phase], ncols=100, desc=phase)
                
                for g, spatial_features, temporal_features, external_features, truth_data in tqdm_loader:
                    features, truth_data = convert_train_truth_to_gpu(
                        [spatial_features, temporal_features, external_features], truth_data)
                    g = convert_to_gpu(g)

                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(g, *features)
                        outputs = torch.squeeze(outputs)
                        loss = loss_func(truth_data, outputs)

                        if phase == 'train':
                            optimizer.zero_grad()
                            loss.backward()
                            optimizer.step()

                    targets.append(truth_data.cpu().numpy())
                    with torch.no_grad():
                        predictions.append(outputs.cpu().numpy())

                    running_loss[phase] += loss.item() * truth_data.size(0)
                    steps += truth_data.size(0)
                    
                    tqdm_loader.set_postfix(loss=running_loss[phase] / steps)

                # End of phase
                scores = evaluate(np.concatenate(predictions), np.concatenate(targets))
                running_metrics[phase] = scores
                print(f"{phase} scores: {scores}")

                if phase == 'validate' and scores['F1-SCORE'] > best_f1_score:
                    best_f1_score = scores['F1-SCORE']
                    save_validate_this_epoch = True
                    save_dict.update(model_state_dict=copy.deepcopy(model.state_dict()),
                                     epoch=epoch,
                                     optimizer_state_dict=copy.deepcopy(optimizer.state_dict()))
                    
                    save_path = f"{model_folder}/model_epoch_{epoch}.pkl"
                    save_model(save_path, **save_dict)
                    print(f"Saved best model to {save_path}")

            scheduler.step(running_loss['train'])

            if save_validate_this_epoch:
                test_metric = running_metrics["test"].copy()

            # Log to Tensorboard
            for metric in running_metrics['train'].keys():
                writer.add_scalars(metric, {f'{phase}': running_metrics[phase][metric] for phase in phases}, global_step=epoch)

    except KeyboardInterrupt:
        print("Training interrupted by user.")
    finally:
        time_elapsed = time.time() - since
        print(f"Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s")
        save_model(f"{model_folder}/best_model.pkl", **save_dict)

    return test_metric


# ==========================================
# 7. MAIN EXECUTION
# ==========================================

def create_model() -> nn.Module:
    spatial_dim = get_attribute('poi_features_number') # 22
    temporal_dim = get_attribute('temporal_features_number') # 1
    external_dim = get_attribute('weather_features_number') + get_attribute('external_temporal_features_number') # 43
    
    model = AI_GFACN(spatial_dim=spatial_dim, 
                     temporal_dim=temporal_dim, 
                     external_dim=external_dim, 
                     hidden_dim=64)
    return model

def create_loss(loss_type):
    if loss_type == 'mse_loss': return MSELoss()
    elif loss_type == 'bce_loss': return BCELoss()
    else: raise ValueError("Unknown loss function.")

if __name__ == '__main__':
    print("Initializing AI-GFACN Training Pipeline...")
    
    # 1. Prepare Data Loaders
    print("Creating data loaders...")
    # NOTE: Ensure speed variable is available or passed correctly
    data_loaders = get_data_loaders(get_attribute('K_hop'), get_attribute('batch_size'))
    print("Data loaders ready.")

    test_metrics = []

    # 2. Training Loop (Repeated runs)
    for train_time in range(get_attribute("train_repeat_times")):
        print(f"\n--- Run {train_time + 1} ---")
        
        model = create_model()
        loss_func = create_loss(loss_type=get_attribute('loss_function'))

        model_folder = f"saves/{get_attribute('model_name')}_run{train_time}"
        tensorboard_folder = f"runs/{get_attribute('model_name')}_run{train_time}"
        
        shutil.rmtree(model_folder, ignore_errors=True)
        os.makedirs(model_folder, exist_ok=True)
        shutil.rmtree(tensorboard_folder, ignore_errors=True)
        os.makedirs(tensorboard_folder, exist_ok=True)

        if get_attribute("optim") == "Adam":
            optimizer = optim.Adam(model.parameters(),
                                   lr=get_attribute("learning_rate"),
                                   weight_decay=get_attribute("weight_decay"))
        else:
            optimizer = optim.SGD(model.parameters(),
                                  lr=get_attribute("learning_rate"),
                                  momentum=0.9)

        test_metric = train_model(model=model,
                                  data_loaders=data_loaders,
                                  loss_func=loss_func,
                                  optimizer=optimizer,
                                  model_folder=model_folder,
                                  tensorboard_folder=tensorboard_folder)
        
        print(f'Run {train_time + 1} Final Test Metric: {test_metric}')
        if test_metric:
            test_metrics.append(test_metric)

    # 3. Final Aggregated Results
    if test_metrics:
        metrics_summary = {}
        for key in test_metrics[0].keys():
            metric_list = [metric[key] for metric in test_metrics]
            metrics_summary[f"mean_{key}"] = float(np.mean(metric_list))
            metrics_summary[f"std_{key}"] = float(np.std(metric_list, ddof=1)) if len(metric_list) > 1 else 0.0

        print('\n--- Final Aggregated Metrics ---')
        print(json.dumps(metrics_summary, indent=4))
        
        # Save results
        with open("final_results.json", "w") as f:
            json.dump(metrics_summary, f, indent=4)
    else:
        print("No metrics generated.")

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)
Using device: cuda:0
Initializing AI-GFACN Training Pipeline...
Creating data loaders...
Loading network from /kaggle/input/dstgcn-dataset/beijing_roadnet.gpickle...
Loading speed data...


Checking missing dates: 100%|██████████| 2208/2208 [00:49<00:00, 44.21it/s]


Filling 168 missing dates...


Filling speed: 100%|██████████| 168/168 [1:36:58<00:00, 34.64s/it]


...

train scores: {'MAE': 0.1189509704709053, 'RMSE': 0.23980397537655132, 'PCC': np.float32(0.877535), 'PRECISION': np.float64(0.9174917491749175), 'RECALL': np.float64(0.9318435754189944), 'F1-SCORE': np.float64(0.9246119733924613), 'AUC': np.float64(0.9754058113042664)}


validate: 100%|█████████████████████████████████████████| 20/20 [00:15<00:00,  1.30it/s, loss=0.203]


validate scores: {'MAE': 0.11922168731689453, 'RMSE': 0.2441113034760448, 'PCC': np.float32(0.87340844), 'PRECISION': np.float64(0.9182389937106918), 'RECALL': np.float64(0.9125), 'F1-SCORE': np.float64(0.9153605015673982), 'AUC': np.float64(0.9744531249999999)}


test: 100%|█████████████████████████████████████████████| 40/40 [00:30<00:00,  1.32it/s, loss=0.235]

test scores: {'MAE': 0.1279377043247223, 'RMSE': 0.2618271759675272, 'PCC': np.float32(0.85236585), 'PRECISION': np.float64(0.9051321928460342), 'RECALL': np.float64(0.910086004691165), 'F1-SCORE': np.float64(0.9076023391812864), 'AUC': np.float64(0.9663622564784717)}
Training complete in 237m 43s
Run 1 Final Test Metric: {'MAE': 0.12825989723205566, 'RMSE': 0.26588823794600175, 'PCC': np.float32(0.8483343), 'PRECISION': np.float64(0.8835666912306559), 'RECALL': np.float64(0.9374511336982018), 'F1-SCORE': np.float64(0.9097116843702581), 'AUC': np.float64(0.9653704119165616)}

--- Final Aggregated Metrics ---
{
    "mean_MAE": 0.12825989723205566,
    "std_MAE": 0.0,
    "mean_RMSE": 0.26588823794600175,
    "std_RMSE": 0.0,
    "mean_PCC": 0.8483343124389648,
    "std_PCC": 0.0,
    "mean_PRECISION": 0.8835666912306559,
    "std_PRECISION": 0.0,
    "mean_RECALL": 0.9374511336982018,
    "std_RECALL": 0.0,
    "mean_F1-SCORE": 0.9097116843702581,
    "std_F1-SCORE": 0.0,
    "mean_AUC"